In [2]:
import pandas as pd
from re import findall

In [3]:
# Olympic Dataframes
olympic_games_df = pd.read_csv(r"Original\Olympic_Dataset\Olympics_Games.csv")
olympic_games_results = pd.read_csv(r"Original\Olympic_Dataset\Olympic_Athlete_Event_Results.csv")
olympic_country_names = pd.read_csv(r"Original\Olympic_Dataset\Olympics_Country.csv")

# Population Dataframe
total_pop_df = pd.read_csv(r"Original\WorldBankData_15-64_Data_Extract_From_Health_Nutrition_and_Population_Statistics\Data.csv")
total_pop_df = total_pop_df.drop(columns=["Series Name", "Series Code"])

In [4]:
# Common Filters
olympic_years = []
for i in range(1964, 2021, 4):
    olympic_years.append(str(i))

### Clean Population Data

In [5]:
# Rename Total Pop Columns
rename_dict = {}
for col in total_pop_df.columns:
    if col not in ("Country Name",  "Country Code"):
        rename_dict[col] = findall(r"\d{4}", col)[0]
total_pop_df.rename(columns=rename_dict, inplace=True)

# Keep Only Summer Olympic Years
cols_to_keep = ["Country Name",  "Country Code"]
cols_to_keep.extend(olympic_years)
total_pop_df = total_pop_df[cols_to_keep]

# Transpose Years
total_pop_df = pd.melt(total_pop_df, id_vars=["Country Name",  "Country Code"]).rename(columns={
    "variable": "Year",
    "value": "Total_Pop"
})

# Drop Country Code
total_pop_df = total_pop_df[total_pop_df['Country Code'] != "AFW"].reset_index(drop=True)

In [6]:
total_pop_df.to_csv("Filtered_Pop_Data.csv")

In [7]:
total_pop_df.head()

,Country Name,Country Code,Year,Total_Pop
0,Africa Eastern and Southern,AFE,1964,64954558
1,Albania,ALB,1964,771925
2,Algeria,DZA,1964,5705901
3,American Samoa,ASM,1964,10820
4,Andorra,AND,1964,3654


### Clean Olympic Data

##### Get Olympic Host

In [8]:
# Drop Winter Games
olympic_games_host = olympic_games_df[olympic_games_df['edition'].str.contains("Summer")].copy()

# Get Common Years
olympic_games_host['year'] = olympic_games_host['year'].astype("str")
olympic_games_host = olympic_games_host[olympic_games_host['year'].isin(olympic_years)].reset_index(drop=True)

# Keep Necessary Columns
olympic_games_host = olympic_games_host[["year", "edition_id", "country_noc"]]

##### Get Olympic Medal Results

In [9]:
# Filter for Games of Interest
olympic_results = olympic_games_results[olympic_games_results['edition_id'].isin(olympic_games_host['edition_id'])].copy()

# Keep only medal positions
olympic_results = olympic_results[olympic_results['medal'].notna()]

# Sort and Reset Index
olympic_results = olympic_results.sort_values(by="edition_id").reset_index(drop=True)

# Drop uneeded columns
olympic_results.drop(columns=["result_id", 'athlete','athlete_id'], inplace=True)

### Merge and Produce Final Chart Datatables

##### Country Medals Broken by Sport

In [10]:
country_event_medals = olympic_results.copy()

# Group Team Sport Medals - ["edition_id", "country_noc", "sport", "event", "pos", "isTeamSport"]
country_event_medals = country_event_medals.drop_duplicates(subset=["edition_id", "country_noc", "sport", "event", "pos", "isTeamSport"])
country_event_medals.drop(columns=['isTeamSport'], inplace=True)

# Get Each Medal Count Grouped by Edition, Country, isTeamSport - (Dont Want to Inflate by Team Sport)
tmp = country_event_medals[["edition", "edition_id", "country_noc", "sport", "event", "medal", "pos"]].groupby(by=["edition", "edition_id", "country_noc", "sport", "event", "medal"]).nunique()
tmp = tmp.reset_index()

# Group Medal Counts into One Table
country_medals_final = country_event_medals[["edition", "edition_id", "country_noc", "sport", "event", "medal"]].copy().drop_duplicates()
country_medals_final = pd.merge(country_medals_final, tmp[tmp['medal']=="Gold"], on=["edition", "edition_id", "country_noc", "sport", "event", "medal"], how="left").rename(columns={"pos":"gold"})
country_medals_final = pd.merge(country_medals_final, tmp[tmp['medal']=="Silver"], on=["edition", "edition_id", "country_noc", "sport", "event", "medal"], how="left").rename(columns={"pos":"silver"})
country_medals_final = pd.merge(country_medals_final, tmp[tmp['medal']=="Bronze"], on=["edition", "edition_id", "country_noc", "sport", "event", "medal"], how="left").rename(columns={"pos":"bronze"})
country_medals_final.fillna(0, inplace=True)

country_medals_final['total'] = country_medals_final['gold'] + country_medals_final['silver'] + country_medals_final['bronze']

# # # Add Overall - Total For All Olympic Games
# # test = country_medals_final.groupby(by=["country_noc", "sport"]).sum().reset_index()
# # test["edition_id"] = "100"
# # test["edition"] = "Overall"
# # country_medals_final = pd.concat([country_medals_final, test])

# Merge in Year
country_medals_final = pd.merge(country_medals_final, olympic_games_df[["edition_id", "year"]], on="edition_id", how="left") 

# Merge in Country Name
country_medals_final = pd.merge(country_medals_final, olympic_country_names, left_on="country_noc", right_on="noc", how="left")
country_medals_final.drop(columns=["noc"], inplace=True)

# Sort by edition_id, country_noc, medal, sport, event
country_medals_final.sort_values(by=["edition_id", "country_noc", "sport", "event"], inplace=True)

# Save Table
country_medals_final.to_csv("Country_Medals_1960_to_2020.csv", index=False)

#### Olympians Versus Population

In [11]:
olympians_v_pop = olympic_games_results[olympic_games_results['edition_id'].isin(olympic_games_host['edition_id'])].copy()
oly_v_pop_final = olympians_v_pop[['edition', 'edition_id', 'country_noc']].copy().rename(columns={
    "country_noc":"Country Code"
    })

# Append Year
oly_v_pop_final['Year'] = oly_v_pop_final['edition'].apply(lambda x: findall(r'\d{4}', x)[0])

# Group by year and athlete
tmp = olympians_v_pop.drop_duplicates(subset=['edition_id', 'country_noc', 'athlete_id'])
tmp = tmp[['edition', 'edition_id', 'country_noc', 'athlete_id']].groupby(by=["edition", 'edition_id', 'country_noc']).count().reset_index().rename(columns={
    'athlete_id': "Num_of_Olympians", 
    "country_noc":"Country Code"
    })
oly_v_pop_final = pd.merge(oly_v_pop_final, tmp, on=['edition', 'edition_id', 'Country Code'], how='left').drop_duplicates()

# Change Total Pop Naming Convention
tmp_total_pop = pd.merge(total_pop_df, olympic_country_names, left_on="Country Name", right_on="country")
tmp_total_pop['Country Code'] = tmp_total_pop['noc']
tmp_total_pop.drop(columns=['country', 'noc'], inplace=True)

# Append Total Pop
tmp = pd.merge(oly_v_pop_final, tmp_total_pop[["Country Code", 'Year', 'Total_Pop']], on=["Country Code", 'Year'])
tmp.sort_values(by=["edition_id", 'Country Code'], inplace=True)
tmp['Total_Pop'] = tmp['Total_Pop'].astype("int64")
tmp['pct_of_pop'] = round(tmp['Num_of_Olympians'] / tmp['Total_Pop'], 8)
oly_v_pop_final = tmp.copy()

# Append Overall
t = tmp[['Country Code', "Num_of_Olympians", "Total_Pop", 'pct_of_pop']].groupby(by=['Country Code']).sum().reset_index()
t['Num_of_Olympians'] = round(t['Num_of_Olympians'], 0)
t.drop(columns=['Total_Pop', 'pct_of_pop'], inplace=True)
# t['Total_Pop'] = round(t['Total_Pop'], 0)
# t['pct_of_pop'] = round(t['pct_of_pop'], 8)

t['edition'] = "Overall"
t['edition_id'] = 100

oly_v_pop_final = pd.concat([oly_v_pop_final, t]).drop(columns=['Year']).reset_index(drop=True)

oly_v_pop_final.to_csv("Olympian_by_Pop.csv", index=False)


### Medals per Olympian Sent

In [12]:
medals_per_olympian_sent = oly_v_pop_final[['edition', 'edition_id', 'Country Code', 'Num_of_Olympians']].copy()
tmp = country_medals_final[['edition_id', 'country_noc', 'gold', 'silver', 'bronze', 'total']].copy().groupby(by=['edition_id', 'country_noc']).sum().reset_index().rename(columns={"country_noc":"Country Code"})

medals_per_olympian_sent['edition_id'] = medals_per_olympian_sent['edition_id'].astype("int64")
tmp['edition_id'] = tmp['edition_id'].astype("int64")

medals_per_olympian_sent = pd.merge(medals_per_olympian_sent, tmp, on=['edition_id', 'Country Code'], how='left')
medals_per_olympian_sent['medal_per_olympian'] = round(medals_per_olympian_sent['total'] / medals_per_olympian_sent['Num_of_Olympians'], 4)
medals_per_olympian_sent.fillna(0, inplace=True)

medals_per_olympian_sent.to_csv("Medals_Per_Olympian.csv", index=False)

In [13]:
country_medals_final[['edition_id', 'country_noc', 'gold', 'silver', 'bronze', 'total']].groupby(by=['edition_id', 'country_noc']).sum().reset_index()

,edition_id,country_noc,gold,silver,bronze,total
0,16,ARG,0.0,1.0,0.0,1.0
1,16,AUS,6.0,2.0,10.0,18.0
2,16,BAH,1.0,0.0,0.0,1.0
3,16,BEL,2.0,0.0,1.0,3.0
4,16,BRA,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...
953,61,UGA,2.0,1.0,1.0,4.0
954,61,UKR,1.0,6.0,12.0,19.0
955,61,USA,39.0,41.0,33.0,113.0
956,61,UZB,3.0,0.0,2.0,5.0


In [14]:
# Need - Edition, edition_id, Host City and Country, Top Performing Country, # of Medals for Country, # of Competing Olympians (Oldest and Youngest Olympian at Event), 

In [133]:
# Get Base info
fun_fact_df = olympic_games_df[["edition", "edition_id", "city", "country_noc", "start_date", "end_date", "year"]].copy()
fun_fact_df = fun_fact_df[fun_fact_df['edition_id'].isin(country_medals_final['edition_id'].unique())]

# Number of Competing Countries
number_of_countries = olympic_games_results[["edition_id", "country_noc"]].drop_duplicates(subset=["edition_id", "country_noc"]).groupby(by=["edition_id"]).count().reset_index().rename(columns={"country_noc":"total_countries"})
fun_fact_df = pd.merge(fun_fact_df, number_of_countries, on=["edition_id"])

# Number of Competitors
number_of_competitors = olympic_games_results[["edition_id", "athlete_id"]].drop_duplicates(subset=["edition_id", "athlete_id"]).groupby(by=["edition_id"]).count().reset_index().rename(columns={"athlete_id":"total_athletes"})
fun_fact_df = pd.merge(fun_fact_df, number_of_competitors, on=["edition_id"])

# Number of Events
number_of_events = olympic_games_results[["edition_id", "event"]].drop_duplicates(subset=["edition_id", "event"]).groupby(by=["edition_id"]).count().reset_index().rename(columns={"event":"total_events"})
fun_fact_df = pd.merge(fun_fact_df, number_of_events, on=["edition_id"])

# Top Performing Country
top_performers = country_medals_final[["edition_id", "country_noc", "total"]].groupby(by=["edition_id", "country_noc"]).sum().reset_index()
top_performers = top_performers.sort_values(by=["edition_id", "total"], ascending=False).drop_duplicates(subset=["edition_id"]).rename(columns={"country_noc":"top_country"})
fun_fact_df = pd.merge(fun_fact_df, top_performers[["edition_id", "top_country", "total"]], on=["edition_id"])

# Add Pop
# tmp_pop = total_pop_df[~total_pop_df.isna().any(axis=1)].copy().rename(columns={
#     'Country Code': "country_noc",
#      "Year": "year"
# }).drop(columns=['Country Name'])
# tmp_pop['year'] = tmp_pop['year'].astype("int64")
# fun_fact_df['year'] = fun_fact_df['year'].astype("int64")
# fun_fact_df = pd.merge(fun_fact_df, tmp_pop, on=['year', "country_noc"], how="left")

# Add Country
fun_fact_df = pd.merge(fun_fact_df, olympic_country_names, left_on=['country_noc'], right_on=["noc"])
fun_fact_df.drop(columns=['noc'], inplace=True)

fun_fact_df.to_csv("fun_fact_df.csv", index=False)

In [141]:
tmp = pd.read_csv(r"C:\Users\Sebastian\OneDrive\Desktop\MCS\CS 416 Data Visualization\Final Project\Data\Clean\fun_fact_df.csv")
tmp = pd.merge(tmp, olympic_country_names, left_on="top_country", right_on="noc")
tmp.to_csv(r"C:\Users\Sebastian\OneDrive\Desktop\MCS\CS 416 Data Visualization\Final Project\Data\Clean\fun_fact_df.csv")

In [140]:
tmp.head()

,edition,edition_id,city,country_noc,start_date,end_date,year,total_countries,total_athletes,total_events,top_country,total,country_x,noc,country_y
0,1964 Summer Olympics,16,Tokyo,JPN,10/10/1964,10/24/1964,1964,93,5422,155,URS,93,Japan,URS,Soviet Union
1,1968 Summer Olympics,17,Ciudad de México,MEX,10/12/1968,10/27/1968,1968,112,5841,172,USA,107,Mexico,USA,United States
2,1972 Summer Olympics,18,München,FRG,8/26/1972,9/11/1972,1972,121,7392,188,URS,99,West Germany,URS,Soviet Union
3,1976 Summer Olympics,19,Montréal,CAN,7/17/1976,8/1/1976,1976,92,6220,183,URS,125,Canada,URS,Soviet Union
4,1980 Summer Olympics,20,Moskva,URS,7/19/1980,8/3/1980,1980,80,5383,189,URS,195,Soviet Union,URS,Soviet Union
